In [12]:
from pathlib import Path
import csv
from feat import Detector

In [ ]:
path_to_save_local = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/outputs/'
path_to_save_server = '/Volumes/LYA/LYA_Lab/FEX_Substudy1/BIDS_dataset/code/FEX_AU_comparison_Marie/FEX_AU_outputs/'

In [ ]:
file_all_folders_server = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/mycode/AU_comparison_software/run_libreface/all_folders.txt'
file_all_folders_local = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/mydata/test_data/all_folders_local.txt'

In [ ]:
def process_pyfeat (which_algorithm, path_all_folders_file, path_to_save):
    
    assert which_algorithm in ['xgb', 'svm']

    print(f'\n\n==============================\n START PYFEAT ({which_algorithm.upper()}) PROCESSING \n=============================\n')
    
    print('\nPrep work...')
    
    if which_algorithm == 'xgb':
        # define paths
        my_path_to_save = path_to_save + 'outputs_pyfeat_xgb'
        # load pyfeat detector module
        detector = Detector(au_model='xgb')
        
    elif which_algorithm == 'svm':
        # define paths
        my_path_to_save = path_to_save + 'outputs_pyfeat_svm'
        # load pyfeat detector module
        detector = Detector(au_model='svm')    

    # prep file to save frames data
    frames_file = f"{my_path_to_save}/frames_pyfeat_{which_algorithm}.csv"
    with open(frames_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            f"name_pf_{which_algorithm}",
            f"total_frames_pf_{which_algorithm}",
            f"processed_frames_pf_{which_algorithm}",])
    
    print('\nPrep work DONE\n')

    # read all folders data, iterate
    with open(path_all_folders_file) as f:
        for line in f:

            # work with data folder by folder:
            
            folderpath = Path(line.strip())
            name_base = (Path(folderpath).name)

            print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
            print(f"\n+++++++++++++++++++++++++++++++++++++\n Processing FOLDER: {name_base} \n+++++++++++++++++++++++++++++++++++++\n")
            print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

            # iterate through all .mp4 files in folder
            for path_to_file in Path(folderpath).iterdir():
                if path_to_file.suffix.lower() == ".mp4": 

                    filename_stem = path_to_file.stem
                    filename = path_to_file.name
                    
                    print(f"\n ----------------------------------- \n Processing file: {filename} \n ----------------------------------- \n")

                    # for this file:
                    
                    # 1. Do pyfeat processing
                    
                    print(f"\n Doing {which_algorithm.upper()} processing...")
                    video_prediction = detector.detect_video(path_to_file, data_type="video")
                    print(f"\n {which_algorithm.upper()} processing: DONE")                    
    
                    # 2. Save pyfeat output
                    
                    print('\n Saving output files...')

                    path_to_save_vid = f"{my_path_to_save}/output_pyfeat_{which_algorithm}_{filename_stem}.csv"            
                    video_prediction.to_csv(path_to_save_vid, index=True)
                    
                    print('\n Saving output files: DONE') 

                    # 3. Save frames stats

                    print('\n Saving frames stats...')
            
                    total_frames_vid = video_prediction.shape[0]
                    processed_frames_vid = video_prediction.FaceScore.isna().value_counts()[False]
            
                    with open(frames_file, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            filename,
                            total_frames_vid,
                            processed_frames_vid
                        ])
                        
                    print('\n Saving frames stats: DONE')

                    print(f"\n ----------------------------------- \n DONE file: {filename} \n ----------------------------------- \n")

            print(f"\n+++++++++++++++++++++++++++++++++++++\n DONE FOLDER: {name_base} \n+++++++++++++++++++++++++++++++++++++\n")
                    
    print(f'\n\n==============================\n DONE PYFEAT ({which_algorithm.upper()}) PROCESSING \n=============================\n')

    return(0)


In [ ]:
# TODO wrap everything in log
process pyfeat
